Berechnung der Markdown

In [3]:
import numpy as np

### Cutler et al. 2021

##### Nachbauen der Ergebnisse
1. Nur statistisch signifikant

In [4]:
'''
Berechnung von log odds: 
Wie sehr sinkt die Wahrscheinlichkeit, dass ein Meeting effizient war, wenn ein bestimmtes Feature der Fall ist. 
Jeder Koeffizient gibt den Effekt dieses Features an. 
Wenn der Koeffizient nur einen anderen Koeffizient, der Effizienz beeinflusst, statistisch signifikant beeinflusst, wird ein odds ratio gebildet. 
'''


'\nBerechnung von log odds: \nWie sehr sinkt die Wahrscheinlichkeit, dass ein Meeting effizient war, wenn ein bestimmtes Feature der Fall ist. \nJeder Koeffizient gibt den Effekt dieses Features an. \nWenn der Koeffizient nur einen anderen Koeffizient, der Effizienz beeinflusst, statistisch signifikant beeinflusst, wird ein odds ratio gebildet. \n'

In [ ]:
def logit(p):
    #log-odds function
    return np.log(p / (1 - p))

def inv_logit(log_odds):
    #Calculates the probability from log-odds
    return 1 / (1 + np.exp(-log_odds))

# --- COEFFICIENTS FROM THE CAUSAL GRAPH (Figure 7) ---
COEFS_INCLUSIVE = {
    'RemoteOnly': 1.6,
    'Participation': 2.5,
    'Video': 1.2,
    'Reliability': 1.7,
    'Quality': 1.7,
    'Agenda': 1.4,
}
B0_INCLUSIVE = 0.0

COEFS_COMFORTABLE = {
    'JoinRemotely': -0.4,
    'Participation': 4.1,
    'Inclusive': 6.7, # Note: This is an intermediate variable input
}
B0_COMFORTABLE = 0.0

COEFS_EFFECTIVE = {
    'Inclusive': 2.2,
    'Comfortable': 3.3,
    'Agenda': 1.4,
}
B0_EFFECTIVE = 0.0



In [6]:

def calculate_meeting_effectiveness_probability(meeting_attributes: dict, B0_E=B0_EFFECTIVE):
    
    required_base_inputs = set([
        'RemoteOnly', 'Participation', 'Video', 'Reliability', 'Quality', 'Agenda', 'JoinRemotely'
    ])
    
    missing_inputs = [key for key in required_base_inputs if key not in meeting_attributes]
    if missing_inputs:
        print(f"ERROR: Missing essential input attributes: {', '.join(missing_inputs)}")
        print("Please provide values (0 or 1) for all root inputs.")
        return None
    
    #Calculate Log-Odds for Inclusive (Intermediate Variable) ---
    log_odds_inclusive = B0_INCLUSIVE
    for attr, coef in COEFS_INCLUSIVE.items():
        log_odds_inclusive += coef * meeting_attributes[attr]
            
    P_inclusive = inv_logit(log_odds_inclusive)
    
    #Convert P(Inclusive) to a predicted binary state (0 or 1) for the next stage
    Predicted_Inclusive = 1 if P_inclusive >= 0.5 else 0 
    
    #Calculate Log-Odds for Comfortable (Intermediate Variable) ---
    log_odds_comfortable = B0_COMFORTABLE
    
    # Inputs from the graph: JoinRemotely, Participation, Inclusive
    log_odds_comfortable += COEFS_COMFORTABLE['JoinRemotely'] * meeting_attributes['JoinRemotely']
    log_odds_comfortable += COEFS_COMFORTABLE['Participation'] * meeting_attributes['Participation']
    log_odds_comfortable += COEFS_COMFORTABLE['Inclusive'] * Predicted_Inclusive 

    P_comfortable = inv_logit(log_odds_comfortable)
    Predicted_Comfortable = 1 if P_comfortable >= 0.5 else 0

    # --- 3. Calculate Log-Odds for Effective (Final Outcome) ---
    log_odds_effective = B0_E
    
    # Input 1: Inclusive (using the predicted binary state)
    log_odds_effective += COEFS_EFFECTIVE['Inclusive'] * Predicted_Inclusive
    
    # Input 2: Comfortable (using the predicted binary state)
    log_odds_effective += COEFS_EFFECTIVE['Comfortable'] * Predicted_Comfortable
    
    # Input 3: Agenda (base attribute)
    log_odds_effective += COEFS_EFFECTIVE['Agenda'] * meeting_attributes['Agenda']

    # --- 4. Calculate Final Probability of Effectiveness ---
    P_effective = inv_logit(log_odds_effective)
    
    print("--- Intermediate Results ---")
    print(f"Log-Odds Inclusive (BX_I): {log_odds_inclusive:.4f}")
    print(f"Predicted P(Inclusive): {P_inclusive:.4f}")
    print(f"Predicted Inclusive State: {Predicted_Inclusive}")
    print("-" * 28)
    print(f"Log-Odds Comfortable (BX_C): {log_odds_comfortable:.4f}")
    print(f"Predicted P(Comfortable): {P_comfortable:.4f}")
    print(f"Predicted Comfortable State: {Predicted_Comfortable}")
    print("-" * 28)
    print(f"Log-Odds Effective (BX_E): {log_odds_effective:.4f}")
    print(f"Final Predicted P(Effective): {P_effective:.4f}")
    print("-" * 28)
    
    return P_effective



In [7]:
## Scenario 1: An Ideal Meeting (High-Potential)
ideal_meeting_attributes = {
    'RemoteOnly': 0, 'Participation': 0, 'Video': 0, 'Reliability': 0, 
    'Quality': 0, 'Agenda': 0, 'JoinRemotely': 0
}
print("## Scenario 1: High-Potential Meeting (Ideal Attributes)")
probability_1 = calculate_meeting_effectiveness_probability(ideal_meeting_attributes)
print(f"Final Probability 1: {probability_1:.4f}")
print("\n" + "="*50 + "\n")

## Scenario 2: A Poor Remote Meeting (Low-Potential)
poor_remote_meeting_attributes = {
    'RemoteOnly': 1, 'Participation': 1, 'Video': 1, 'Reliability': 1, 
    'Quality': 1, 'Agenda': 1, 'JoinRemotely': 1
}
print("## Scenario 2: Low-Potential Meeting (Poor Remote Attributes)")
probability_2 = calculate_meeting_effectiveness_probability(poor_remote_meeting_attributes)
print(f"Final Probability 2: {probability_2:.4f}")

## Scenario 1: High-Potential Meeting (Ideal Attributes)
--- Intermediate Results ---
Log-Odds Inclusive (BX_I): 0.0000
Predicted P(Inclusive): 0.5000
Predicted Inclusive State: 1
----------------------------
Log-Odds Comfortable (BX_C): 6.7000
Predicted P(Comfortable): 0.9988
Predicted Comfortable State: 1
----------------------------
Log-Odds Effective (BX_E): 5.5000
Final Predicted P(Effective): 0.9959
----------------------------
Final Probability 1: 0.9959


## Scenario 2: Low-Potential Meeting (Poor Remote Attributes)
--- Intermediate Results ---
Log-Odds Inclusive (BX_I): 10.1000
Predicted P(Inclusive): 1.0000
Predicted Inclusive State: 1
----------------------------
Log-Odds Comfortable (BX_C): 10.4000
Predicted P(Comfortable): 1.0000
Predicted Comfortable State: 1
----------------------------
Log-Odds Effective (BX_E): 6.9000
Final Predicted P(Effective): 0.9990
----------------------------
Final Probability 2: 0.9990


2. Statistisch signifikant und nicht signifikant

In [ ]:
def inv_logit(log_odds):
    """Calculates the probability (inverse logit function: P = 1 / (1 + e^(-LogOdds)))."""
    return 1 / (1 + np.exp(-log_odds))


GLM_COEFFICIENTS = {
    # 1. Model for Participation (Input for Inclusive and Comfortable)
    # Participation = f(RemoteOnly, Comfortable, MeetingSize5p, JoinFromRemotely, Postmeet)
    'Participation': {
        'RemoteOnly': 0.42,
        'Comfortable': 1.51,
        'MeetingSize5p': -1.34,
        'JoinFromRemotely': -0.76,
        'Postmeet': 0.31,
        'B0': 0.0 # Assumed Intercept
    },
    
    # 2. Model for Inclusive (Intermediate Variable)
    # Inclusive = f(RemoteOnly, Participation, Comfortable, MeetingSize5p, Video, Agenda, Quality, Reliability)
    'Inclusive': {
        'RemoteOnly': 0.39,
        'Participation': 0.29,
        'Comfortable': 1.89,
        'MeetingSize5p': -0.50,
        'Video': 0.21,
        'Agenda': 0.43,
        'Quality': 0.49,
        'Reliability': 0.47,
        'B0': 0.0 # Assumed Intercept
    },
    
    # 3. Model for Comfortable (Intermediate Variable)
    # Comfortable = f(RemoteOnly, Participation, MeetingSize5p, JoinFromRemotely, Agenda, Postmeet, Reliability)
    'Comfortable': {
        'RemoteOnly': 0.45,
        'Participation': 1.51,
        'MeetingSize5p': -0.13,
        'JoinFromRemotely': -0.46,
        'Agenda': 0.42,
        'Postmeet': 0.23,
        'Reliability': 0.47,
        'B0': 0.0 # Assumed Intercept
    },
    
    # 4. Model for Effective (Final Outcome)
    # Effective = f(RemoteOnly, Inclusive, Participation, Comfortable, MeetingSize5p, JoinFromRemotely, Video, Agenda, Premeet, Postmeet, Quality, Reliability)
    'Effective': {
        'RemoteOnly': -0.04,
        'Inclusive': 0.77,
        'Participation': 0.00,
        'Comfortable': 1.21,
        'MeetingSize5p': 0.01,
        'JoinFromRemotely': 0.18,
        'Video': 0.11,
        'Agenda': 0.26,
        'Premeet': 0.14,
        'Postmeet': 0.08,
        'Quality': 0.15,
        'Reliability': 0.11,
        'B0': 0.0 # Assumed Intercept
    }
}


def calculate_meeting_effectiveness_probability_v2(meeting_attributes: dict):
    """
    Calculates the probability of Effective using GLM coefficients from Table 5
    and cascading predicted probabilities (continuous values) for intermediate variables.
    """
    
    # --- 0. Define all required root inputs (from Table 5) ---
    required_root_inputs = set([
        'RemoteOnly', 'MeetingSize5p', 'JoinFromRemotely', 'Video', 'Agenda', 
        'Premeet', 'Postmeet', 'Quality', 'Reliability'
    ])
    
    # Check for missing required inputs
    missing_inputs = [key for key in required_root_inputs if key not in meeting_attributes]
    if missing_inputs:
        print(f"ERROR: Missing essential root attributes: {', '.join(missing_inputs)}")
        print("Please provide values (0 or 1) for all root inputs defined in the table.")
        return None

    
    # --- 1. Calculate Participation (P_participation) ---
    # This must be calculated first as it feeds into Inclusive and Comfortable.
    log_odds_part = GLM_COEFFICIENTS['Participation']['B0']
    for attr, coef in GLM_COEFFICIENTS['Participation'].items():
        if attr in meeting_attributes:
            log_odds_part += coef * meeting_attributes[attr]
            
    P_participation = inv_logit(log_odds_part)
    
    # --- 2. Calculate Inclusive (P_inclusive) ---
    # Depends on root inputs AND P_participation, P_comfortable (circular dependency, handle Comfortable later)
    # Note: In a proper GLM/PACM implementation, P_comfortable would be either predicted 
    # simultaneously or iteratively. For this linear chain, we'll calculate Inclusive and Comfortable 
    # based on the assumption that the *other* is a root input for now.
    
    # We will treat Comfortable's input on Inclusive as a zero for now, 
    # as the table shows a circular dependency (Inclusive->Comfortable and Comfortable->Inclusive).
    # Since the graph does not show a direct Comfortable->Inclusive edge, we will ignore the 1.89** for now.
    
    log_odds_inclusive = GLM_COEFFICIENTS['Inclusive']['B0']
    for attr, coef in GLM_COEFFICIENTS['Inclusive'].items():
        if attr in meeting_attributes:
            log_odds_inclusive += coef * meeting_attributes[attr]
        elif attr == 'Participation':
            log_odds_inclusive += coef * P_participation # Use continuous probability
        # Ignoring 'Comfortable' input to break the circular dependency for this simple script
    
    P_inclusive = inv_logit(log_odds_inclusive)

    
    # --- 3. Calculate Comfortable (P_comfortable) ---
    # Depends on root inputs AND P_participation
    
    log_odds_comfortable = GLM_COEFFICIENTS['Comfortable']['B0']
    for attr, coef in GLM_COEFFICIENTS['Comfortable'].items():
        if attr in meeting_attributes:
            log_odds_comfortable += coef * meeting_attributes[attr]
        elif attr == 'Participation':
            log_odds_comfortable += coef * P_participation # Use continuous probability
    
    P_comfortable = inv_logit(log_odds_comfortable)

    
    # --- 4. Calculate Effective (P_effective) ---
    # Depends on root inputs AND P_inclusive, P_participation, P_comfortable
    
    log_odds_effective = GLM_COEFFICIENTS['Effective']['B0']
    for attr, coef in GLM_COEFFICIENTS['Effective'].items():
        if attr in meeting_attributes:
            log_odds_effective += coef * meeting_attributes[attr]
        elif attr == 'Inclusive':
            log_odds_effective += coef * P_inclusive # Use continuous probability
        elif attr == 'Participation':
            log_odds_effective += coef * P_participation # Use continuous probability
        elif attr == 'Comfortable':
            log_odds_effective += coef * P_comfortable # Use continuous probability

    P_effective = inv_logit(log_odds_effective)

    # --- Print Intermediate Results (Continuous) ---
    print("--- Intermediate Results (Table 5 Coefficients) ---")
    print(f"Log-Odds Participation (BX_P): {log_odds_part:.4f}")
    print(f"Predicted P(Participation): {P_participation:.4f}")
    print("-" * 45)
    print(f"Log-Odds Inclusive (BX_I): {log_odds_inclusive:.4f}")
    print(f"Predicted P(Inclusive): {P_inclusive:.4f}")
    print("-" * 45)
    print(f"Log-Odds Comfortable (BX_C): {log_odds_comfortable:.4f}")
    print(f"Predicted P(Comfortable): {P_comfortable:.4f}")
    print("-" * 45)
    print(f"Log-Odds Effective (BX_E): {log_odds_effective:.4f}")
    print(f"Final Predicted P(Effective): {P_effective:.4f}")
    print("-" * 45)
    
    return P_effective


# --- RE-DEFINING INPUT SCENARIOS FOR TABLE 5 ---

## SCENARIO 1: Ideal Meeting (Maximum Positive Inputs)
# Note: JoinFromRemotely is the new variable from the table, replacing JoinRemotely.
ideal_meeting_attributes_v2 = {
    'RemoteOnly': 0, # In-person/Hybrid preferred (-0.04 to Effective)
    'MeetingSize5p': 0, # Small meeting preferred (Coeff -0.5 on Inclusive, -1.34 on Participation)
    'JoinFromRemotely': 0, # Attending in-person preferred (Coeff -0.76 on Participation, -0.46 on Comfortable)
    'Video': 1, # Video used
    'Agenda': 1, # Agenda used
    'Premeet': 1, # Premeet used
    'Postmeet': 1, # Postmeet used
    'Quality': 1, # High Quality
    'Reliability': 1, # High Reliability
}

print("## Scenario 1: High-Potential Meeting (Using Table 5 Coefficients)")
probability_1_v2 = calculate_meeting_effectiveness_probability_v2(ideal_meeting_attributes_v2)
print(f"Final Probability 1: {probability_1_v2:.4f}")

print("\n" + "="*70 + "\n")

## SCENARIO 2: Poor Remote Meeting (Maximum Negative/Minimizing Positive Inputs)
poor_remote_meeting_attributes_v2 = {
    'RemoteOnly': 1, # Remote-only
    'MeetingSize5p': 1, # Large meeting size
    'JoinFromRemotely': 1, # Joining remotely
    'Video': 0, # No video
    'Agenda': 0, # No agenda
    'Premeet': 0, # No premeet
    'Postmeet': 0, # No postmeet
    'Quality': 0, # Low quality
    'Reliability': 0, # Low reliability
}

print("## Scenario 2: Low-Potential Meeting (Using Table 5 Coefficients)")
probability_2_v2 = calculate_meeting_effectiveness_probability_v2(poor_remote_meeting_attributes_v2)
print(f"Final Probability 2: {probability_2_v2:.4f}")

## Scenario 1: High-Potential Meeting (Using Table 5 Coefficients)
--- Intermediate Results (Table 5 Coefficients) ---
Log-Odds Participation (BX_P): 0.3100
Predicted P(Participation): 0.5769
---------------------------------------------
Log-Odds Inclusive (BX_I): 1.7673
Predicted P(Inclusive): 0.8541
---------------------------------------------
Log-Odds Comfortable (BX_C): 1.9911
Predicted P(Comfortable): 0.8799
---------------------------------------------
Log-Odds Effective (BX_E): 2.5723
Final Predicted P(Effective): 0.9291
---------------------------------------------
Final Probability 1: 0.9291


## Scenario 2: Low-Potential Meeting (Using Table 5 Coefficients)
--- Intermediate Results (Table 5 Coefficients) ---
Log-Odds Participation (BX_P): -1.6800
Predicted P(Participation): 0.1571
---------------------------------------------
Log-Odds Inclusive (BX_I): -0.0644
Predicted P(Inclusive): 0.4839
---------------------------------------------
Log-Odds Comfortable (BX_C): 0.0972
Pre

### Hosseinkashi et al. 2024
##### Nachbauen der Ergebnisse
1. statistich signifikant

In [34]:
import numpy as np
import math # Import the math module for ln

def inv_logit(log_odds):
    """Calculates the probability (inverse logit function: P = 1 / (1 + e^(-LogOdds)))."""
    return 1 / (1 + np.exp(-log_odds))

# --- COEFFICIENTS FROM TABLE 3 (Combined Model, only statistically significant ORs) ---
# Odds Ratios (OR) were converted to GLM Coefficients (beta_i = ln(OR)).
# Intercepts (B0) are NOT provided and are ASSUMED to be 0 for all models.

GLM_COEFFICIENTS_V3 = {
    'Participation': {
        'Reliability Issues': math.log(0.13), # -2.040
        'Recurring': math.log(0.82),          # -0.199
        'ScreenShare': math.log(0.71),        # -0.342
        # WICHTIG: Die korrekte Bezeichnung aus der Tabelle
        'Small Meeting (8 Or Less)': math.log(7.13), # 1.964 
        'Short Call (10min. or less)': math.log(0.72),# -0.329
        'Headset': math.log(1.16),            # 0.148
        'Video Duration > 30%': math.log(1.17), # 0.157
        'B0': 0.0
    },
    
    'Inclusive': {
        'Quality Issues': math.log(0.35),      # -1.050
        'Reliability Issues': math.log(0.49),  # -0.713
        'Participation': math.log(4.05),       # 1.399
        # WICHTIG: Die korrekte Bezeichnung aus der Tabelle
        'Small Meeting (8 Or Less)': math.log(1.51), # 0.412
        'Short Call (10min. or less)': math.log(0.61),# -0.494
        'B0': 0.0
    },
    
    'Effective': {
        'Quality Issues': math.log(0.14),      # -1.966
        'Inclusive': math.log(45.48),          # 3.817
        'ScreenShare': math.log(1.39),         # 0.330
        # WICHTIG: Die korrekte Bezeichnung aus der Tabelle
        'Small Meeting (8 Or Less)': math.log(1.29), # 0.255
        'B0': 0.0
    }
}


def calculate_meeting_effectiveness_probability_v3(meeting_attributes: dict):
    """
    Calculates the probability of Effective using the *significant* GLM coefficients
    derived from Table 3 Odds Ratios, passing continuous probabilities.
    """
    
    # --- 0. Define all required root inputs ---
    required_root_inputs = set(
        list(GLM_COEFFICIENTS_V3['Participation'].keys()) +
        list(GLM_COEFFICIENTS_V3['Inclusive'].keys()) +
        list(GLM_COEFFICIENTS_V3['Effective'].keys())
    )
    # Entferne die berechneten Variablen und Intercepts
    required_root_inputs.difference_update(['B0', 'Participation', 'Inclusive']) 
    
    missing_inputs = [key for key in required_root_inputs if key not in meeting_attributes]
    if missing_inputs:
        print(f"ERROR: Missing essential root attributes: {', '.join(missing_inputs)}")
        print("Please provide values (0 or 1) for all root inputs defined in the model.")
        return None

    
    # --- 1. Calculate Participation (P_participation) ---
    log_odds_part = GLM_COEFFICIENTS_V3['Participation']['B0']
    for attr, coef in GLM_COEFFICIENTS_V3['Participation'].items():
        if attr in meeting_attributes:
            log_odds_part += coef * meeting_attributes[attr]
            
    P_participation = inv_logit(log_odds_part)
    
    # --- 2. Calculate Inclusive (P_inclusive) ---
    log_odds_inclusive = GLM_COEFFICIENTS_V3['Inclusive']['B0']
    for attr, coef in GLM_COEFFICIENTS_V3['Inclusive'].items():
        if attr in meeting_attributes:
            log_odds_inclusive += coef * meeting_attributes[attr]
        elif attr == 'Participation':
            log_odds_inclusive += coef * P_participation 
    
    P_inclusive = inv_logit(log_odds_inclusive)

    
    # --- 3. Calculate Effective (P_effective) ---
    log_odds_effective = GLM_COEFFICIENTS_V3['Effective']['B0']
    for attr, coef in GLM_COEFFICIENTS_V3['Effective'].items():
        if attr in meeting_attributes:
            log_odds_effective += coef * meeting_attributes[attr]
        elif attr == 'Inclusive':
            log_odds_effective += coef * P_inclusive 

    P_effective = inv_logit(log_odds_effective)

    # --- Print Intermediate Results (Continuous) ---
    print("--- Intermediate Results (Tabelle 3 Koeffizienten, nur signifikant) ---")
    print(f"Log-Odds Participation (BX_P): {log_odds_part:.4f}")
    print(f"Predicted P(Participation): {P_participation:.4f}")
    print("-" * 65)
    print(f"Log-Odds Inclusive (BX_I): {log_odds_inclusive:.4f}")
    print(f"Predicted P(Inclusive): {P_inclusive:.4f}")
    print("-" * 65)
    print(f"Log-Odds Effective (BX_E): {log_odds_effective:.4f}")
    print(f"Final Predicted P(Effective): {P_effective:.4f}")
    print("-" * 65)
    
    return P_effective

# --- KORRIGIERTE SZENARIO-DEFINITIONEN (Fix des 'Or' vs 'or' Fehlers) ---

## SCENARIO 1: Ideal Meeting (Optimal)
ideal_meeting_attributes_v3 = {
    # Alle negativen Koeffizienten auf 0 setzen
    'Reliability Issues': 0, 
    'Quality Issues': 0,     
    'Recurring': 0,
    'ScreenShare': 0,
    'Short Call (10min. or less)': 0,
    
    # Alle positiven Koeffizienten auf 1 setzen
    'Small Meeting (8 Or Less)': 1, # FIX: Großes 'Or'
    'Headset': 1,
    'Video Duration > 30%': 1,
}

print("## Szenario 1: Optimales Meeting (Tabelle 3)")
probability_1_v3 = calculate_meeting_effectiveness_probability_v3(ideal_meeting_attributes_v3)
print(f"Final Probability 1: {probability_1_v3:.4f}")

print("\n" + "="*75 + "\n")

## SCENARIO 2: Poor Meeting (Schlecht)
poor_meeting_attributes_v3 = {
    # Alle negativen Koeffizienten auf 1 setzen
    'Reliability Issues': 1, 
    'Quality Issues': 1,     
    'Recurring': 1,
    'ScreenShare': 1,
    'Short Call (10min. or less)': 1,
    
    # Alle positiven Koeffizienten auf 0 setzen
    'Small Meeting (8 Or Less)': 0, # FIX: Großes 'Or'
    'Headset': 0,
    'Video Duration > 30%': 0,
}

print("## Szenario 2: Schlechtes Meeting (Tabelle 3)")
probability_2_v3 = calculate_meeting_effectiveness_probability_v3(poor_meeting_attributes_v3)
print(f"Final Probability 2: {probability_2_v3:.4f}")

## Szenario 1: Optimales Meeting (Tabelle 3)
--- Intermediate Results (Tabelle 3 Koeffizienten, nur signifikant) ---
Log-Odds Participation (BX_P): 2.2697
Predicted P(Participation): 0.9063
-----------------------------------------------------------------
Log-Odds Inclusive (BX_I): 1.6798
Predicted P(Inclusive): 0.8429
-----------------------------------------------------------------
Log-Odds Effective (BX_E): 3.4721
Final Predicted P(Effective): 0.9699
-----------------------------------------------------------------
Final Probability 1: 0.9699


## Szenario 2: Schlechtes Meeting (Tabelle 3)
--- Intermediate Results (Tabelle 3 Koeffizienten, nur signifikant) ---
Log-Odds Participation (BX_P): -2.9097
Predicted P(Participation): 0.0517
-----------------------------------------------------------------
Log-Odds Inclusive (BX_I): -2.1852
Predicted P(Inclusive): 0.1011
-----------------------------------------------------------------
Log-Odds Effective (BX_E): -1.2509
Final Predicted P(Eff

2. nicht statistisch signifikant

In [31]:
import numpy as np

def inv_logit(log_odds):
    """Calculates the probability (inverse logit function: P = 1 / (1 + e^(-LogOdds)))."""
    return 1 / (1 + np.exp(-log_odds))

def or_to_beta(odds_ratio):
    """Converts an Odds Ratio (OR) to a GLM Coefficient (beta) using the natural logarithm."""
    return np.log(odds_ratio)

# --- COEFFICIENTS FROM TABLE 3 (Adjusted Odds Ratios -> converted to Beta) ---
# NOTE: Intercepts (B0) are NOT provided and are ASSUMED to be 0 for all models.

# Table 3 lists Odds Ratios for:
# Effective = f(Quality Issues, Inclusive, ScreenShare, Small Meeting)
# Inclusive = f(Quality Issues, Reliability Issues, Participation, Small Meeting, Short Call)
# Participation = f(Reliability Issues, Recurring, ScreenShare, Small Meeting, Short Call, Headset, Video Duration)

# 1. Model for Participation
# Participation = f(Reliability Issues, Recurring, ScreenShare, Small Meeting, Short Call, Headset, Video Duration > 30%)
COEFS_PARTICIPATION = {
    'Reliability Issues': or_to_beta(0.13), # Negative influence
    'Recurring': or_to_beta(0.82), # Negative influence
    'ScreenShare': or_to_beta(0.71), # Negative influence
    'Small Meeting (8 or less)': or_to_beta(7.13), # Strong positive influence
    'Short Call (10min. or less)': or_to_beta(0.72), # Negative influence
    'Headset': or_to_beta(1.16), # Positive influence
    'Video Duration > 30%': or_to_beta(1.17), # Positive influence
    'B0': 0.0 # Assumed Intercept
}

# 2. Model for Inclusive
# Inclusive = f(Quality Issues, Reliability Issues, Participation, Small Meeting, Short Call)
COEFS_INCLUSIVE = {
    'Quality Issues': or_to_beta(0.35), # Negative influence
    'Reliability Issues': or_to_beta(0.44), # Negative influence
    'Participation': or_to_beta(4.05), # Strong positive influence (used as P_participation)
    'Small Meeting (8 or less)': or_to_beta(1.51), # Positive influence
    'Short Call (10min. or less)': or_to_beta(0.61), # Negative influence
    'B0': 0.0 # Assumed Intercept
}

# 3. Model for Effective (Final Outcome)
# Effective = f(Quality Issues, Inclusive, ScreenShare, Small Meeting)
COEFS_EFFECTIVE = {
    'Quality Issues': or_to_beta(0.14), # Strong negative influence
    'Inclusive': or_to_beta(45.48), # **Extremely Strong positive influence** (used as P_inclusive)
    'ScreenShare': or_to_beta(1.39), # Positive influence
    'Small Meeting (8 or less)': or_to_beta(1.29), # Positive influence
    'B0': 0.0 # Assumed Intercept
}


def calculate_meeting_effectiveness_probability_v3(meeting_attributes: dict):
    """
    Calculates the probability of Effective using Adjusted Odds Ratios from Table 3,
    converted to GLM coefficients (beta_i = ln(OR)), and cascading probabilities.
    """
    
    # --- Define all required root inputs based on the union of all model inputs in Table 3 ---
    required_root_inputs = set(
        list(COEFS_PARTICIPATION.keys()) +
        ['Quality Issues'] + ['Reliability Issues'] +
        ['Short Call (10min. or less)'] + ['Headset'] + ['Video Duration > 30%'] +
        ['Recurring'] + ['ScreenShare']
    )
    required_root_inputs.discard('B0')
    required_root_inputs.discard('Participation') # These are intermediate outcomes
    required_root_inputs.discard('Inclusive')

    # Check for missing required inputs
    missing_inputs = [key for key in required_root_inputs if key not in meeting_attributes]
    if missing_inputs:
        print(f"ERROR: Missing essential root attributes: {', '.join(missing_inputs)}")
        print("Please provide values (0 or 1) for all root inputs defined in Table 3.")
        return None

    
    # --- 1. Calculate Participation (P_participation) ---
    log_odds_part = COEFS_PARTICIPATION['B0']
    for attr, coef in COEFS_PARTICIPATION.items():
        if attr in meeting_attributes:
            log_odds_part += coef * meeting_attributes[attr]
            
    P_participation = inv_logit(log_odds_part)
    
    # --- 2. Calculate Inclusive (P_inclusive) ---
    log_odds_inclusive = COEFS_INCLUSIVE['B0']
    for attr, coef in COEFS_INCLUSIVE.items():
        if attr in meeting_attributes:
            log_odds_inclusive += coef * meeting_attributes[attr]
        elif attr == 'Participation':
            log_odds_inclusive += coef * P_participation # Use continuous probability
    
    P_inclusive = inv_logit(log_odds_inclusive)

    
    # --- 3. Calculate Effective (P_effective) ---
    log_odds_effective = COEFS_EFFECTIVE['B0']
    for attr, coef in COEFS_EFFECTIVE.items():
        if attr in meeting_attributes:
            log_odds_effective += coef * meeting_attributes[attr]
        elif attr == 'Inclusive':
            log_odds_effective += coef * P_inclusive # Use continuous probability

    P_effective = inv_logit(log_odds_effective)

    # --- Print Intermediate Results (Continuous) ---
    print("--- Intermediate Results (Table 3 ORs -> Beta Coefficients) ---")
    print(f"Log-Odds Participation (BX_P): {log_odds_part:.4f}")
    print(f"Predicted P(Participation): {P_participation:.4f}")
    print("-" * 60)
    print(f"Log-Odds Inclusive (BX_I): {log_odds_inclusive:.4f}")
    print(f"Predicted P(Inclusive): {P_inclusive:.4f}")
    print("-" * 60)
    print(f"Log-Odds Effective (BX_E): {log_odds_effective:.4f}")
    print(f"Final Predicted P(Effective): {P_effective:.4f}")
    print("-" * 60)
    
    return P_effective


# --- SCENARIOS DEFINED FOR TABLE 3 VARIABLES ---

# Note on New Variables:
# - 'Small Meeting (8 or less)' is POSITIVE (OR=7.13 to Participation)
# - 'Quality Issues', 'Reliability Issues' are NEGATIVE (OR < 1)

## SCENARIO 1: Ideal Meeting (Maximize Positive Inputs)
ideal_meeting_attributes_v3 = {
    # Maximize Participation
    'Reliability Issues': 0,        # No Issues (OR 0.13 is negative)
    'Recurring': 0,                 # Not Recurring (OR 0.82 is negative)
    'ScreenShare': 0,               # No Screen Share (OR 0.71 is negative)
    'Small Meeting (8 or less)': 1, # Yes, Small Meeting (OR 7.13 is strong positive)
    'Short Call (10min. or less)': 0, # Not Short Call (OR 0.72 is negative)
    'Headset': 1,                   # Yes, Headset (OR 1.16 is positive)
    'Video Duration > 30%': 1,      # Yes, long video (OR 1.17 is positive)
    
    # Maximize Effective (and Inclusive)
    'Quality Issues': 0,            # No Issues (OR 0.14 is strong negative)
}

print("## Scenario 1: High-Potential Meeting (Using Table 3 Adjusted ORs)")
probability_1_v3 = calculate_meeting_effectiveness_probability_v3(ideal_meeting_attributes_v3)
print(f"Final Probability 1: {probability_1_v3:.4f}")

print("\n" + "="*70 + "\n")

## SCENARIO 2: Poor Meeting (Maximize Negative Inputs)
poor_meeting_attributes_v3 = {
    # Minimize Participation
    'Reliability Issues': 1,        # Yes, Reliability Issues (OR 0.13 is negative)
    'Recurring': 1,                 # Yes, Recurring (OR 0.82 is negative)
    'ScreenShare': 1,               # Yes, Screen Share (OR 0.71 is negative)
    'Small Meeting (8 or less)': 0, # No, Large Meeting (OR 7.13 is strong positive)
    'Short Call (10min. or less)': 1, # Yes, Short Call (OR 0.72 is negative)
    'Headset': 0,                   # No Headset (OR 1.16 is positive)
    'Video Duration > 30%': 0,      # No, short video (OR 1.17 is positive)
    
    # Minimize Effective (and Inclusive)
    'Quality Issues': 1,            # Yes, Quality Issues (OR 0.14 is strong negative)
}

print("## Scenario 2: Low-Potential Meeting (Using Table 3 Adjusted ORs)")
probability_2_v3 = calculate_meeting_effectiveness_probability_v3(poor_meeting_attributes_v3)
print(f"Final Probability 2: {probability_2_v3:.4f}")

## Scenario 1: High-Potential Meeting (Using Table 3 Adjusted ORs)
--- Intermediate Results (Table 3 ORs -> Beta Coefficients) ---
Log-Odds Participation (BX_P): 2.2697
Predicted P(Participation): 0.9063
------------------------------------------------------------
Log-Odds Inclusive (BX_I): 1.6798
Predicted P(Inclusive): 0.8429
------------------------------------------------------------
Log-Odds Effective (BX_E): 3.4721
Final Predicted P(Effective): 0.9699
------------------------------------------------------------
Final Probability 1: 0.9699


## Scenario 2: Low-Potential Meeting (Using Table 3 Adjusted ORs)
--- Intermediate Results (Table 3 ORs -> Beta Coefficients) ---
Log-Odds Participation (BX_P): -2.9097
Predicted P(Participation): 0.0517
------------------------------------------------------------
Log-Odds Inclusive (BX_I): -2.2928
Predicted P(Inclusive): 0.0917
------------------------------------------------------------
Log-Odds Effective (BX_E): -1.2867
Final Predicted P(Ef

In [39]:
import numpy as np
import math

# --- FUNKTIONEN UND KOEFFIZIENTEN (Basierend auf Tabelle 3, nur signifikant) ---

def inv_logit(log_odds):
    """Calculates the probability (inverse logit function: P = 1 / (1 + e^(-LogOdds)))."""
    return 1 / (1 + np.exp(-log_odds))

# GLM COEFFICIENTS (Odds Ratios aus Tabelle 3 umgerechnet zu ln(OR) und auf Signifikanz gefiltert)
GLM_COEFFICIENTS_V3 = {
    # Partizipation: Beta-Werte (ln(OR))
    'Participation': {
        'Reliability Issues': math.log(0.13), # -2.040
        'Recurring': math.log(0.82),          # -0.199
        'ScreenShare': math.log(0.71),        # -0.342
        'Small Meeting (8 Or Less)': math.log(7.13), # 1.964
        'Short Call (10min. or less)': math.log(0.72),# -0.329
        'Headset': math.log(1.16),            # 0.148
        'Video Duration > 30%': math.log(1.17), # 0.157
        'B0': 0.0
    },
    # Inklusiv: Beta-Werte (ln(OR))
    'Inclusive': {
        'Quality Issues': math.log(0.35),      # -1.050
        'Reliability Issues': math.log(0.49),  # -0.713
        'Participation': math.log(4.05),       # 1.399
        'Small Meeting (8 Or Less)': math.log(1.51), # 0.412
        'Short Call (10min. or less)': math.log(0.61),# -0.494
        'B0': 0.0
    },
    # Effektiv: Beta-Werte (ln(OR))
    'Effective': {
        'Quality Issues': math.log(0.14),      # -1.966
        'Inclusive': math.log(45.48),          # 3.817
        'ScreenShare': math.log(1.39),         # 0.330
        'Small Meeting (8 Or Less)': math.log(1.29), # 0.255
        'B0': 0.0
    }
}

# Mapping für lesbare Ausgaben und Einfluss-Interpretation
COEFFICIENT_DESCRIPTIONS = {
    'Reliability Issues': 'Zuverlässigkeitsprobleme (stark negativ)',
    'Recurring': 'Wiederkehrendes Meeting (leicht negativ)',
    'ScreenShare': 'Bildschirmfreigabe (negativ)',
    'Small Meeting (8 Or Less)': 'Kleine Gruppe (<= 8 Personen, stark positiv)',
    'Short Call (10min. or less)': 'Kurzes Meeting (<= 10 Min, negativ)',
    'Headset': 'Headset benutzt (positiv)',
    'Video Duration > 30%': 'Lange Videonutzung (>30%, positiv)',
    'Quality Issues': 'Qualitätsprobleme (sehr stark negativ)',
}


def calculate_effectiveness_metrics(meeting_attributes):
    """Führt die kaskadierte Berechnung durch."""
    
    # 1. Participation
    log_odds_part = GLM_COEFFICIENTS_V3['Participation']['B0']
    for attr, coef in GLM_COEFFICIENTS_V3['Participation'].items():
        if attr in meeting_attributes:
            log_odds_part += coef * meeting_attributes[attr]
            
    P_participation = inv_logit(log_odds_part)
    
    # 2. Inclusive
    log_odds_inclusive = GLM_COEFFICIENTS_V3['Inclusive']['B0']
    for attr, coef in GLM_COEFFICIENTS_V3['Inclusive'].items():
        if attr in meeting_attributes:
            log_odds_inclusive += coef * meeting_attributes[attr]
        elif attr == 'Participation':
            log_odds_inclusive += coef * P_participation 
    
    P_inclusive = inv_logit(log_odds_inclusive)

    
    # 3. Effective
    log_odds_effective = GLM_COEFFICIENTS_V3['Effective']['B0']
    for attr, coef in GLM_COEFFICIENTS_V3['Effective'].items():
        if attr in meeting_attributes:
            log_odds_effective += coef * meeting_attributes[attr]
        elif attr == 'Inclusive':
            log_odds_effective += coef * P_inclusive 

    P_effective = inv_logit(log_odds_effective)
    
    return {
        'P_participation': P_participation, 'P_inclusive': P_inclusive, 'P_effective': P_effective,
        'log_odds_part': log_odds_part, 'log_odds_inclusive': log_odds_inclusive, 'log_odds_effective': log_odds_effective,
    }


def generate_effectiveness_report(scenario_name: str, meeting_attributes: dict) -> str:
    """
    Generiert einen Markdown-formatierten Bericht, der direkt exportiert werden kann.
    """
    
    try:
        metrics = calculate_effectiveness_metrics(meeting_attributes)
    except Exception as e:
        return f"Fehler bei der Berechnung für Szenario {scenario_name}: {e}"
    
    report = []
    
    # --- I. BERICHTSÜBERSCHRIFT ---
    report.append(f"### Meeting-Effizienz-Analyse: {scenario_name}")
    report.append("-" * 50)
    
    # --- II. ERGEBNIS-ZUSAMMENFASSUNG ---
    report.append("#### Zusammenfassung")
    report.append(f"Die berechnete Wahrscheinlichkeit, dass das Meeting **effektiv** ist: **{metrics['P_effective']:.2%}**")
    report.append(f"*(Log-Odds (BX): {metrics['log_odds_effective']:.4f})*")
    report.append("\n")
    
    # --- III. DETAILLIERTE FAKTOREN UND EINGABEN ---
    report.append("#### Verwendete Faktoren (Input und Wert)")
    report.append("Die folgenden Faktoren (0 = Nein/Schlecht, 1 = Ja/Gut) wurden dem Modell zugeführt:")
    report.append("\n")
    report.append("| Faktor | Wert (0/1) | Beschreibung (Einfluss) |")
    report.append("| :--- | :--- | :--- |")
    
    input_factors = sorted([key for key in meeting_attributes.keys()])
    
    for attr in input_factors:
        value = meeting_attributes.get(attr)
        description = COEFFICIENT_DESCRIPTIONS.get(attr, 'Direkter Root-Faktor')
        report.append(f"| {attr} | **{value}** | {description} |")
    
    report.append("\n")

    # --- IV. KASKADIERTE ZWISCHENERGEBNISSE ---
    report.append("#### Zwischenergebnisse")
    report.append("Das Modell berechnet die Wahrscheinlichkeit stufenweise:")
    report.append("\n")
    report.append("| Zielvariable | Log-Odds (BX) | Wahrscheinlichkeit (P) |")
    report.append("| :--- | :--- | :--- |")
    report.append(f"| **1. Participation** | {metrics['log_odds_part']:.4f} | {metrics['P_participation']:.2%} |")
    report.append(f"| **2. Inclusive** | {metrics['log_odds_inclusive']:.4f} | {metrics['P_inclusive']:.2%} |")
    report.append(f"| **3. Effective (Final)** | {metrics['log_odds_effective']:.4f} | **{metrics['P_effective']:.2%}** |")
    report.append("\n")
    
    # --- V. INTERPRETATION ---
    if metrics['P_effective'] >= 0.8:
        interpretation = "Das Meeting hat eine **sehr hohe** Erfolgswahrscheinlichkeit (Log-Odds > 1.38). Die positiven Faktoren dominieren klar. Insbesondere der starke Einfluss von 'Inclusive' (OR 45.48) treibt das Ergebnis nach oben."
    elif metrics['P_effective'] >= 0.5:
        interpretation = "Das Meeting hat eine **knapp positive** Erfolgswahrscheinlichkeit (Log-Odds zwischen 0 und 1.38). Eine leichte Verbesserung der Partizipations- oder Inklusivitäts-Faktoren könnte das Ergebnis deutlich über 50% heben."
    else:
        interpretation = "Das Meeting hat eine **niedrige** Erfolgswahrscheinlichkeit (Log-Odds < 0). Die negativen Faktoren ('Quality Issues', 'Reliability Issues') sind dominant und müssen dringend behoben werden."
        
    report.append("#### Interpretation")
    report.append(interpretation)
    
    return "\n".join(report)

# --- ANWENDUNG DER SZEANRIEN ---

# Scenario 1 (Optimal) Attributes
ideal_meeting_attributes_v3 = {
    'Reliability Issues': 1, 'Quality Issues': 0, 'Recurring': 0,
    'ScreenShare': 1, 'Short Call (10min. or less)': 0,
    'Small Meeting (8 Or Less)': 1, 
    'Headset': 1,
    'Video Duration > 30%': 1,
}

# Scenario 2 (Schlecht) Attributes
poor_meeting_attributes_v3 = {
    'Reliability Issues': 1, 'Quality Issues': 1, 'Recurring': 1,
    'ScreenShare': 1, 'Short Call (10min. or less)': 1,
    'Small Meeting (8 Or Less)': 0, 
    'Headset': 0,
    'Video Duration > 30%': 0,
}


# 1. Bericht für Szenario 1 generieren
report_s1 = generate_effectiveness_report("Optimales Meeting (S1)", ideal_meeting_attributes_v3)
print(report_s1)

print("\n" + "="*75 + "\n")

# 2. Bericht für Szenario 2 generieren
report_s2 = generate_effectiveness_report("Schlechtes Meeting (S2)", poor_meeting_attributes_v3)
print(report_s2)

### Meeting-Effizienz-Analyse: Optimales Meeting (S1)
--------------------------------------------------
#### Zusammenfassung
Die berechnete Wahrscheinlichkeit, dass das Meeting **effektiv** ist: **94.43%**
*(Log-Odds (BX): 2.8312)*


#### Verwendete Faktoren (Input und Wert)
Die folgenden Faktoren (0 = Nein/Schlecht, 1 = Ja/Gut) wurden dem Modell zugeführt:


| Faktor | Wert (0/1) | Beschreibung (Einfluss) |
| :--- | :--- | :--- |
| Headset | **1** | Headset benutzt (positiv) |
| Quality Issues | **0** | Qualitätsprobleme (sehr stark negativ) |
| Recurring | **0** | Wiederkehrendes Meeting (leicht negativ) |
| Reliability Issues | **1** | Zuverlässigkeitsprobleme (stark negativ) |
| ScreenShare | **1** | Bildschirmfreigabe (negativ) |
| Short Call (10min. or less) | **0** | Kurzes Meeting (<= 10 Min, negativ) |
| Small Meeting (8 Or Less) | **1** | Kleine Gruppe (<= 8 Personen, stark positiv) |
| Video Duration > 30% | **1** | Lange Videonutzung (>30%, positiv) |


#### Zwischenergebn

### Meeting-Effizienz-Analyse: Optimales Meeting (S1)
--------------------------------------------------
#### Zusammenfassung
Die berechnete Wahrscheinlichkeit, dass das Meeting **effektiv** ist: **96.99%**
*(Log-Odds (BX): 3.4721)*


#### Verwendete Faktoren (Input und Wert)
Die folgenden Faktoren (0 = Nein/Schlecht, 1 = Ja/Gut) wurden dem Modell zugeführt:


| Faktor | Wert (0/1) | Beschreibung (Einfluss) |
| :--- | :--- | :--- |
| Headset | **1** | Headset benutzt (positiv) |
| Quality Issues | **0** | Qualitätsprobleme (sehr stark negativ) |
| Recurring | **0** | Wiederkehrendes Meeting (leicht negativ) |
| Reliability Issues | **0** | Zuverlässigkeitsprobleme (stark negativ) |
| ScreenShare | **0** | Bildschirmfreigabe (negativ) |
| Short Call (10min. or less) | **0** | Kurzes Meeting (<= 10 Min, negativ) |
| Small Meeting (8 Or Less) | **1** | Kleine Gruppe (<= 8 Personen, stark positiv) |
| Video Duration > 30% | **1** | Lange Videonutzung (>30%, positiv) |


#### Zwischenergebnisse
Das Modell berechnet die Wahrscheinlichkeit stufenweise:


| Zielvariable | Log-Odds (BX) | Wahrscheinlichkeit (P) |
| :--- | :--- | :--- |
| **1. Participation** | 2.2697 | 90.63% |
| **2. Inclusive** | 1.6798 | 84.29% |
| **3. Effective (Final)** | 3.4721 | **96.99%** |


#### Interpretation
Das Meeting hat eine **sehr hohe** Erfolgswahrscheinlichkeit (Log-Odds > 1.38). Die positiven Faktoren dominieren klar. Insbesondere der starke Einfluss von 'Inclusive' (OR 45.48) treibt das Ergebnis nach oben.

===========================================================================

### Meeting-Effizienz-Analyse: Schlechtes Meeting (S2)
--------------------------------------------------
#### Zusammenfassung
Die berechnete Wahrscheinlichkeit, dass das Meeting **effektiv** ist: **22.25%**
*(Log-Odds (BX): -1.2509)*


#### Verwendete Faktoren (Input und Wert)
Die folgenden Faktoren (0 = Nein/Schlecht, 1 = Ja/Gut) wurden dem Modell zugeführt:


| Faktor | Wert (0/1) | Beschreibung (Einfluss) |
| :--- | :--- | :--- |
| Headset | **0** | Headset benutzt (positiv) |
| Quality Issues | **1** | Qualitätsprobleme (sehr stark negativ) |
| Recurring | **1** | Wiederkehrendes Meeting (leicht negativ) |
| Reliability Issues | **1** | Zuverlässigkeitsprobleme (stark negativ) |
| ScreenShare | **1** | Bildschirmfreigabe (negativ) |
| Short Call (10min. or less) | **1** | Kurzes Meeting (<= 10 Min, negativ) |
| Small Meeting (8 Or Less) | **0** | Kleine Gruppe (<= 8 Personen, stark positiv) |
| Video Duration > 30% | **0** | Lange Videonutzung (>30%, positiv) |


#### Zwischenergebnisse
Das Modell berechnet die Wahrscheinlichkeit stufenweise:


| Zielvariable | Log-Odds (BX) | Wahrscheinlichkeit (P) |
| :--- | :--- | :--- |
| **1. Participation** | -2.9097 | 5.17% |
| **2. Inclusive** | -2.1852 | 10.11% |
| **3. Effective (Final)** | -1.2509 | **22.25%** |


#### Interpretation
Das Meeting hat eine **niedrige** Erfolgswahrscheinlichkeit (Log-Odds < 0). Die negativen Faktoren ('Quality Issues', 'Reliability Issues') sind dominant und müssen dringend behoben werden.

### Meeting-Effizienz-Analyse: Optimales Meeting
--------------------------------------------------
#### Zusammenfassung
Die berechnete Wahrscheinlichkeit, dass das Meeting **effektiv** ist: **96.99%**


#### Verwendete Faktoren (Input und Wert)
Die folgenden Faktoren (0 = Nein/Schlecht, 1 = Ja/Gut) wurden dem Modell zugeführt:


| Faktor | Wert (0/1) | Beschreibung (Einfluss) |
| :--- | :--- | :--- |
| Headset | **1** | Headset benutzt (positiv) |
| Quality Issues | **0** | Qualitätsprobleme (sehr stark negativ) |
| Recurring | **0** | Wiederkehrendes Meeting (leicht negativ) |
| Reliability Issues | **0** | Zuverlässigkeitsprobleme (stark negativ) |
| ScreenShare | **0** | Bildschirmfreigabe (negativ) |
| Short Call (10min. or less) | **0** | Kurzes Meeting (<= 10 Min, negativ) |
| Small Meeting (8 Or Less) | **1** | Kleine Gruppe (<= 8 Personen, stark positiv) |
| Video Duration > 30% | **1** | Lange Videonutzung (>30%, positiv) |


#### Zwischenergebnisse
Das Modell berechnet die Wahrscheinlichkeit stufenweise:

| Zielvariable | Log-Odds (BX) | Wahrscheinlichkeit (P) |
| :--- | :--- | :--- |
| **1. Participation** | 2.2697 | 90.63% |
| **2. Inclusive** | 1.6798 | 84.29% |
| **3. Effective (Final)** | 3.4721 | **96.99%** |


===========================================================================

### Meeting-Effizienz-Analyse: Schlechtes Meeting 
--------------------------------------------------
#### Zusammenfassung
Die berechnete Wahrscheinlichkeit, dass das Meeting **effektiv** ist: **22.25%**


#### Verwendete Faktoren (Input und Wert)
Die folgenden Faktoren (0 = Nein/Schlecht, 1 = Ja/Gut) wurden dem Modell zugeführt:


| Faktor | Wert (0/1) | Beschreibung (Einfluss) |
| :--- | :--- | :--- |
| Headset | **0** | Headset benutzt (positiv) |
| Quality Issues | **1** | Qualitätsprobleme (sehr stark negativ) |
| Recurring | **1** | Wiederkehrendes Meeting (leicht negativ) |
| Reliability Issues | **1** | Zuverlässigkeitsprobleme (stark negativ) |
| ScreenShare | **1** | Bildschirmfreigabe (negativ) |
| Short Call (10min. or less) | **1** | Kurzes Meeting (<= 10 Min, negativ) |
| Small Meeting (8 Or Less) | **0** | Kleine Gruppe (<= 8 Personen, stark positiv) |
| Video Duration > 30% | **0** | Lange Videonutzung (>30%, positiv) |


#### Zwischenergebnisse
Das Modell berechnet die Wahrscheinlichkeit stufenweise:


| Zielvariable | Log-Odds (BX) | Wahrscheinlichkeit (P) |
| :--- | :--- | :--- |
| **1. Participation** | -2.9097 | 5.17% |
| **2. Inclusive** | -2.1852 | 10.11% |
| **3. Effective (Final)** | -1.2509 | **22.25%** |


